QAP-Ising transformation is based on this paper: https://arxiv.org/abs/1811.11538

### Generate QAP Hamiltonian

Note: this notebook generates Ising-type Hamiltonian with quadratic terms only for a given QAP. 

In [1]:
# import all necessary libraries

import numpy as np
from sympy import *
import re

In [2]:
# define F (facility matrix) and D (distance matrix)
F = np.array([[0, 5, 2],
              [5, 0, 3],
              [2, 3, 0]])

D = np.array([[0, 8, 15],
              [8, 0, 13],
              [15, 13, 0]])

matrix_size = 3

# penalty coefficient
P = 200

In [3]:
# generate z_dictionary for QAP

number_of_variables = matrix_size*matrix_size
variable_indices = np.arange(0, number_of_variables)

z_dictionary = {}
index = 0

for i in range(matrix_size):
    for j in range(matrix_size):
        z_dictionary[str(i)+str(j)] = "Z" + str(variable_indices[index])
        index = index + 1

#print(z_dictionary)

In [4]:
# generate dictionary with ZZ-pairs and coefficients

quadratic_problem_dict = {}
counter = 0

for i in range(matrix_size):
    for j in range(matrix_size):
        for k in range(matrix_size):
            for l in range(matrix_size):
                
                if F[i, j]*D[k, l] != 0:
                    
                    quadratic_problem_dict[z_dictionary[str(i)+str(k)] + "*" + z_dictionary[str(j)+str(l)]] = F[i, j]*D[k, l]

#print(quadratic_problem_dict)

In [5]:
# simplify the indices taking symmetry into account

i=0
quadratic_problem_dict_keys = list(quadratic_problem_dict.keys())
quadratic_problem_parsed_dict = {}

while i < len(quadratic_problem_dict_keys):
    
    #print(quadratic_problem_dict_keys)
    
    key = quadratic_problem_dict_keys[i]
    key1 = str(key[3:5] + "*" + key[0:2])
    
    if key1 in quadratic_problem_dict:
        quadratic_problem_parsed_dict[key] = quadratic_problem_dict[key] + quadratic_problem_dict[key1]
        quadratic_problem_dict_keys.remove(key1)
        
    i = i+1 
    
#print(quadratic_problem_parsed_dict)

In [6]:
### generate problem Hamiltonian

problem_ham = ""

for key in quadratic_problem_parsed_dict:
    
    problem_ham = problem_ham + " + " + str(quadratic_problem_parsed_dict[key]) + "*" + str(key)
    
print(problem_ham)

 + 80*Z0*Z4 + 150*Z0*Z5 + 80*Z1*Z3 + 130*Z1*Z5 + 150*Z2*Z3 + 130*Z2*Z4 + 32*Z0*Z7 + 60*Z0*Z8 + 32*Z1*Z6 + 52*Z1*Z8 + 60*Z2*Z6 + 52*Z2*Z7 + 48*Z3*Z7 + 90*Z3*Z8 + 48*Z4*Z6 + 78*Z4*Z8 + 90*Z5*Z6 + 78*Z5*Z7


In [7]:
### generate penalty part (sum over any colum or row = 1)

array_of_indices = np.zeros((matrix_size, matrix_size))
index = 0

for i in range(matrix_size):
    for j in range(matrix_size):
        array_of_indices[i, j] = variable_indices[index]
        index = index + 1
    
#print(array_of_indices)

penalty = ""

# penalties over colums
for i in range(matrix_size):
    penalty = penalty + "+ " + str(P) + "*(Z" + str( int(array_of_indices[0, i])) + "+Z" + str( int(array_of_indices[1, i])) + "+Z" + str( int(array_of_indices[2, i])) + "-1)**2 "

    
# penalties over rows
for i in range(matrix_size):
    penalty = penalty + "+ " + str(P) + "*(Z" + str( int(array_of_indices[i, 0])) + "+Z" + str( int(array_of_indices[i, 1])) + "+Z" + str( int(array_of_indices[i, 2])) + "-1)**2 "

print(penalty)
    

+ 200*(Z0+Z3+Z6-1)**2 + 200*(Z1+Z4+Z7-1)**2 + 200*(Z2+Z5+Z8-1)**2 + 200*(Z0+Z1+Z2-1)**2 + 200*(Z3+Z4+Z5-1)**2 + 200*(Z6+Z7+Z8-1)**2 


In [8]:
total_ham = expand(problem_ham + penalty)
print(str(total_ham))

400*Z0**2 + 400*Z0*Z1 + 400*Z0*Z2 + 400*Z0*Z3 + 80*Z0*Z4 + 150*Z0*Z5 + 400*Z0*Z6 + 32*Z0*Z7 + 60*Z0*Z8 - 800*Z0 + 400*Z1**2 + 400*Z1*Z2 + 80*Z1*Z3 + 400*Z1*Z4 + 130*Z1*Z5 + 32*Z1*Z6 + 400*Z1*Z7 + 52*Z1*Z8 - 800*Z1 + 400*Z2**2 + 150*Z2*Z3 + 130*Z2*Z4 + 400*Z2*Z5 + 60*Z2*Z6 + 52*Z2*Z7 + 400*Z2*Z8 - 800*Z2 + 400*Z3**2 + 400*Z3*Z4 + 400*Z3*Z5 + 400*Z3*Z6 + 48*Z3*Z7 + 90*Z3*Z8 - 800*Z3 + 400*Z4**2 + 400*Z4*Z5 + 48*Z4*Z6 + 400*Z4*Z7 + 78*Z4*Z8 - 800*Z4 + 400*Z5**2 + 90*Z5*Z6 + 78*Z5*Z7 + 400*Z5*Z8 - 800*Z5 + 400*Z6**2 + 400*Z6*Z7 + 400*Z6*Z8 - 800*Z6 + 400*Z7**2 + 400*Z7*Z8 - 800*Z7 + 400*Z8**2 - 800*Z8 + 1200


total_ham now contains linear terms. If we want to construct QUBO matrix we should have only quadratic terms. Remember that x takes 0 or 1. So we can use the equation x=x2=xx. https://blueqat.readthedocs.io/en/stable/ising.html 

Basically, this transformation will maintain quadratic terms and change the total energy shift after removal of identity terms.

In [9]:
# transform linear terms to identity terms and remove them (total energy shift)

total_ham_string = str(total_ham)

splitted_total_ham = total_ham_string.split(" ")

for j in range(len(splitted_total_ham)):
    
    Z_terms = [m.start() for m in re.finditer('Z', splitted_total_ham[j])]
    
    if len(Z_terms)==1:
        
        for index in variable_indices:  

            marker1 = splitted_total_ham[j].find("*Z"+ str(index)) # linear term
            if marker1 != -1:
                splitted_total_ham[j] = splitted_total_ham[j][0 : marker1 : ]
                    
            marker2 = splitted_total_ham[j].find("*Z"+ str(index)+"**2") # quadratic term
            if marker2 != -1:
                splitted_total_ham[j] = splitted_total_ham[j][0 : marker2 : ]

        #print(splitted_total_ham[j])
        
#print(splitted_total_ham)
        
# assemble the eleents back to Hamiltonian string

total_ham_string = ""

for k in range(len(splitted_total_ham)):
    
    total_ham_string = total_ham_string + " " + splitted_total_ham[k]
                    
print(total_ham_string)      

 400 + 400*Z0*Z1 + 400*Z0*Z2 + 400*Z0*Z3 + 80*Z0*Z4 + 150*Z0*Z5 + 400*Z0*Z6 + 32*Z0*Z7 + 60*Z0*Z8 - 800 + 400 + 400*Z1*Z2 + 80*Z1*Z3 + 400*Z1*Z4 + 130*Z1*Z5 + 32*Z1*Z6 + 400*Z1*Z7 + 52*Z1*Z8 - 800 + 400 + 150*Z2*Z3 + 130*Z2*Z4 + 400*Z2*Z5 + 60*Z2*Z6 + 52*Z2*Z7 + 400*Z2*Z8 - 800 + 400 + 400*Z3*Z4 + 400*Z3*Z5 + 400*Z3*Z6 + 48*Z3*Z7 + 90*Z3*Z8 - 800 + 400 + 400*Z4*Z5 + 48*Z4*Z6 + 400*Z4*Z7 + 78*Z4*Z8 - 800 + 400 + 90*Z5*Z6 + 78*Z5*Z7 + 400*Z5*Z8 - 800 + 400 + 400*Z6*Z7 + 400*Z6*Z8 - 800 + 400 + 400*Z7*Z8 - 800 + 400 - 800 + 1200


In [10]:
# # remove identity terms

# #total_ham_string = str(total_ham)

# marker = 0

# while marker!=-1:

#     marker = total_ham_string.find("**2")
    
#     if marker != -1:
#         if len(total_ham_string) > marker:
#             total_ham_string = total_ham_string[0 : marker - 3 : ] + total_ham_string[marker + 3 : :]
        
# print(total_ham_string)        

In [11]:
# simplify
total_ham = expand(total_ham_string)

In [12]:
# prepare for qbOS
total_ham_string = str(total_ham).replace("*", " ")
print(total_ham_string)

400 Z0 Z1 + 400 Z0 Z2 + 400 Z0 Z3 + 80 Z0 Z4 + 150 Z0 Z5 + 400 Z0 Z6 + 32 Z0 Z7 + 60 Z0 Z8 + 400 Z1 Z2 + 80 Z1 Z3 + 400 Z1 Z4 + 130 Z1 Z5 + 32 Z1 Z6 + 400 Z1 Z7 + 52 Z1 Z8 + 150 Z2 Z3 + 130 Z2 Z4 + 400 Z2 Z5 + 60 Z2 Z6 + 52 Z2 Z7 + 400 Z2 Z8 + 400 Z3 Z4 + 400 Z3 Z5 + 400 Z3 Z6 + 48 Z3 Z7 + 90 Z3 Z8 + 400 Z4 Z5 + 48 Z4 Z6 + 400 Z4 Z7 + 78 Z4 Z8 + 90 Z5 Z6 + 78 Z5 Z7 + 400 Z5 Z8 + 400 Z6 Z7 + 400 Z6 Z8 + 400 Z7 Z8 - 2400


Note: The total_ham_string contains not only the quadratic, but also linear terms

### Run QaoaSimple

Note: To use the extended set of parameters one should count the number of Hamiltonian terms

In [13]:
import qbos_op
qa = qbos_op.QaoaSimple()
qa.qn = number_of_variables
qa.ham = total_ham_string

qa.acc='qpp'
qa.functol[0][0][0]=1e-5
qa.maxeval=800
qa.qaoa_step=10
qa.theta[0][0]=qbos_op.ND()

# # use standard set of parameters
# qa.extended_param = False
# for ii in range(qa.qaoa_step[0][0]*2) :
#         qa.theta[0][0][ii] = 0.25
        
# use extended set of parameters        
qa.extended_param = True
for ii in range((qa.qn[0][0] + 36)*qa.qaoa_step[0][0]): 
    qa.theta[0][0][ii] = 0.25

qa.run()
print('cost ' + str(qa.out_energy[0][0][0]))
print('eigenstate ' + str(qa.out_eigenstate[0][0]))

cost -2935.984375
eigenstate 011000010


### Run QaoaRecursive

In [14]:
print("* Runs recursive QAOA on built-in Nelder-Mead. Uses qpp and QAOA-depth = 5 with extended parameter set. Assert checks the optimum eigenstate.")
import time
import qbos_op

qa = qbos_op.QaoaRecursive()
qa.qaoa_step = 3
qa.sn = 1024   
qa.extended_param = True

# specify the number of qubits
qa.qn = number_of_variables

# specify the threshold number of variables (terms in final Hamiltonian)
qa.n_c = 5

qa.ham = total_ham_string
qa.maxeval = 800

tic = time.perf_counter()
qa.run()
toc = time.perf_counter()

minutes = (toc-tic)/60
print(f"Algorithm runs {minutes:0.2f} minutes")

* Runs recursive QAOA on built-in Nelder-Mead. Uses qpp and QAOA-depth = 5 with extended parameter set. Assert checks the optimum eigenstate.
Algorithm runs 5.39 minutes


In [15]:
print('cost ' + str(qa.out_energy[0][0][0]))
print('eigenstate ' + str(qa.out_eigenstate[0][0]))

cost -910.77734375
eigenstate 100101010
